In [1]:
import pandas as pd
import numpy as np
import os, os.path
import pickle

### 6th column refers to coverage

In [2]:
# features (peaks)
ENCODE_cCREs = pd.read_table("../../../data/ENCODE_cCREs/ENCOCE_cREs_merged_names.tsv", sep='\t', header=None)


In [3]:
ENCODE_cCREs.shape

(926535, 1)

In [4]:
ENCODE_cCREs = pd.read_table("/project/scATAC_analysis/10X_PBMC_analysis/AnnotationQuestion/overlap_bulk_and_scATAC/sc_multiome/10X_PBMC_multiome/data_10X_PBMC_multiome_granulocytes_removed_ARC_1.0.0/ENCODE_cCRE_coverage/ENCOCE_cREs_merged_names.tsv", sep='\t', header=None)


## columns --> number of bulk data

In [5]:
corces_cell_types = ["pDC"]
path_dir_base = "../../../data/Corces2016_bulk_ATAC/01_ENCODE_cCRE_coverages/pDC/"
ENCODE_cCRE_overlapped_bedFiles_dir=[]
SRR_IDs=[]
# convert to a pandas df for the easy access to incex etc
for cell_type in corces_cell_types:
    for bed_file in os.listdir(path_dir_base):
        if bed_file.endswith("_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed"):
            #print(f)
            ENCODE_cCRE_overlapped_bedFiles_dir.append(bed_file)
            SRR_IDs.append(bed_file.split('_')[0])

In [6]:
ENCODE_cCRE_overlapped_bedFiles_dir

['SRR7650770_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed',
 'SRR7650811_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed',
 'SRR7650916_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed']

In [7]:
# number of bulk atac data --> number of columns
num_ENCODE_cCRE_overlapped_bedFiles =  len(ENCODE_cCRE_overlapped_bedFiles_dir)
num_ENCODE_cCRE_overlapped_bedFiles

3

## create bulk_by_ENCODE_peaks matrix

In [8]:
corces_cell_types = ["pDC"]
path_dir_base = "../../../data/Corces2016_bulk_ATAC/01_ENCODE_cCRE_coverages/pDC/"
bulk_by_ENCODE_peaks = np.zeros((len(ENCODE_cCREs),0))
# convert to a pandas df for the easy access to incex etc
bulk_by_ENCODE_peaks_df = pd.DataFrame(bulk_by_ENCODE_peaks,index=ENCODE_cCREs[0])
for cell_type in corces_cell_types:
    path_dir = path_dir_base 
    for bed_file in os.listdir(path_dir):
        if bed_file.endswith("_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed"):
            print(bed_file)
            SRR_name = bed_file.split('_')[0]
            full_path = path_dir +"/"+ bed_file
            SRR_df = pd.read_table(full_path, sep='\t', header=None)
            SRR_df['merged_peak_name']=SRR_df[0].astype(str)+'_'+SRR_df[1].astype(str)+'_'+SRR_df[2].astype(str)
            SRR_df.index = SRR_df.loc[:,"merged_peak_name"]
            SRR_df = pd.DataFrame(SRR_df.loc[:,6])
            SRR_df.columns = [SRR_name]
            bulk_by_ENCODE_peaks_df= pd.concat([bulk_by_ENCODE_peaks_df,SRR_df], axis=1)

SRR7650770_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed
SRR7650811_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed
SRR7650916_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed


In [9]:
bulk_by_ENCODE_peaks_df_annotated = bulk_by_ENCODE_peaks_df.copy()

In [10]:
bulk_by_ENCODE_peaks_df_annotated.columns= ["pDC_1", "pDC_2","pDC_3"]

In [11]:
bulk_by_ENCODE_peaks_df_annotated.to_csv('../../../data/Corces2016_bulk_ATAC/02_ENCODE_coverage_by_prototypes_matrix/Calderon2019_pDC_ENCODE_cCRE_overlappingPeaks_annotated.csv')

In [12]:
# Saving the objects:
with open('../../../data/Corces2016_bulk_ATAC/02_ENCODE_coverage_by_prototypes_matrix/Calderon2019_pDC_ENCODE_cCRE_overlappingPeaks_annotated.pkl', 'wb') as f:
    pickle.dump(bulk_by_ENCODE_peaks_df_annotated, f)